# Περιεχόμενα

## Setup

In [1]:
# Files to be used

# Paths for csv
fcrime = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
fstations = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv"
fincome = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv'
fcodes = 's3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv'

# Paths for .parquet
fcrime_parq = 's3://groups-bucket-dblab-905418150721/group10/CrimeData.parquet'
fstations_parq = 's3://groups-bucket-dblab-905418150721/group10/PoliceStations.parquet'

# Paths for GeoJSON
fgeo = 's3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson'
fgeofields = 's3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks_fields.csv'

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
254,application_1738075734771_0255,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Imports

from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import year, when, count, sum, col, row_number, to_timestamp, regexp_replace
from pyspark.sql.window import Window
import time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Create the spark session
from sedona.spark import *
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Advanced DB") \
    .config('spark.executor.instances','4') \
    .getOrCreate()

sedona = SedonaContext.create(spark)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Schemas

In [4]:
# Crimes table

crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("DateRptd", DateType()),
    StructField("DATEOCC", DateType()),
    StructField("TIMEOCC", StringType()),
    StructField("AREA", StringType()),
    StructField("AREANAME", StringType()),
    StructField("RptDistNo", StringType()),
    StructField("Part", IntegerType()),
    StructField("CrmCd", StringType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", StringType()),
    StructField("VictSex", StringType()),
    StructField("VictDescent", StringType()),
    StructField("PremisCd", StringType()),
    StructField("PremisDesc", StringType()),
    StructField("WeaponUsedCd", StringType()),
    StructField("WeaponDesc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("CrmCd1", StringType()),
    StructField("CrmCd2", StringType()),
    StructField("CrmCd3", StringType()),
    StructField("CrmCd4", StringType()),
    StructField("LOCATION", StringType()),
    StructField("CrossStreet", StringType()),
    StructField("LAT", FloatType()),
    StructField("LON", FloatType()),
])

crimes_df = spark.read.csv(fcrime, header=True, schema=crimes_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Stations table

stations_schema = StructType([
    StructField("X", FloatType()),
    StructField("Y", FloatType()),
    StructField("FID", IntegerType()),
    StructField("DIVISION", StringType()),
    StructField("LOCATION", StringType()),
    StructField("PREC", IntegerType()),
])

stations_df = spark.read.csv(fstations, header=True, schema=stations_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Query 1

### Να υλοποιηθεί το __Query 1__ χρησιμοποιώντας τα DataFrame και RDD APIs. Να εκτελέσετε και τις δύο υλοποιήσεις με 4 Spark executors. Υπάρχει διαφορά στην επίδοση μεταξύ των δύο APIs; Αιτιολογήσετε την απάντησή σας.

#### DataFrame API

In [6]:
# Create the spark session
spark = spark.newSession() \
    .builder \
    .appName("Query 1") \
    .config('spark.executor.instances','4') \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Start timer
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Categorize into age groups
categorized_df = crimes_df.withColumn('age_group',
                    when(col('Vict Age').cast('int') < 18, 'Children')
                    .when(
                        ((col('Vict Age').cast('int') >= 18) & (col('Vict Age').cast('int') <= 24)), 'Young Adults'
                        )
                    .when(
                        ((col('Vict Age').cast('int') >= 25) & (col('Vict Age').cast('int') <= 64)), 'Adults'
                        )
                    .when(col('Vict Age').cast('int') > 64, 'Elderly')
                    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Filter for 'AGGRAVATED ASSAULT'
assault_df = categorized_df.filter(
    col('Crm Cd Desc').contains('AGGRAVATED ASSAULT')
    ) \
    .groupby('age_group') \
    .agg(count('*').alias('victim_count')) \
    .orderBy(col('victim_count').desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Show results
assault_df.show()

time_end = time.time()
time_df = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+
|   age_group|victim_count|
+------------+------------+
|      Adults|       72610|
|Young Adults|       23472|
|    Children|       10724|
|     Elderly|        3099|
+------------+------------+

#### RDD API

Δυστυχώς στα δεδομένα μας έχουμε μερικές περιπτώσεις όπου υπάρχει υποδιαστολή εντός quote marks (""), δημιουργώντας έτσι θέμα στο parse.
Για αυτό θα χρησιμοποιήσουμε μια βιβλιοθήκη της Python για την ανάγνωση των csv αρχείων και θα την περάσουμε μέσω mapping σε κάθε δεδομένο:

In [11]:
import csv
from datetime import datetime

def parse_line(line):
    reader = csv.reader([line])
    fields = next(reader)
    return {
        'DR_NO': fields[0],
        'DateRptd': fields[1],
        'DATEOCC': fields[2],
        'TIMEOCC': fields[3],
        'AREA': fields[4],
        'AREANAME': fields[5],
        'RptDistNo': fields[6],
        'Part': fields[7],
        'CrmCd': fields[8],
        'CrmCdDesc': fields[9],
        'Mocodes': fields[10],
        'VictAge': int(fields[11]),
        'VictSex': fields[12],
        'VictDescent': fields[13],
        'PremisCd': fields[14],
        'PremisDesc': fields[15],
        'WeaponUsedCd': fields[16],
        'WeaponDesc': fields[17],
        'Status': fields[18],
        'CrmCd1': fields[19],
        'CrmCd2': fields[20],
        'CrmCd3': fields[21],
        'CrmCd4': fields[22],
        'LOCATION': fields[23],
        'CrossStreet': fields[24],
        'LAT': fields[25],
        'LON': fields[26]
    }


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Read the .csv as Text
rdd = spark.sparkContext.textFile(fcrime)

# Remove the header
header = rdd.first()
crimes_rdd = rdd.filter(lambda line: line != header).map(parse_line)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Start timer
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Map function to categorize age groups
def categorize_age(crime):
    age = crime['VictAge']
    if age < 18:
        return 'Children'
    elif age >= 18 and age <= 24:
        return 'Young Adults'
    elif age >= 25 and age <= 64:
        return 'Adults'
    else:
        return 'Elderly'
    
categorized_rdd = crimes_rdd.filter(lambda x: 'AGGRAVATED ASSAULT' in x['CrmCdDesc']) \
                    .map(lambda x: (categorize_age(x), 1)) \
                    .reduceByKey(lambda a, b: a + b) \
                    .sortBy(lambda x: -x[1])

categorized_rdd.collect()
time_end = time.time()
time_rdd = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Παρατηρήσεις:

Η διαφορά στην απόδοση μεταξύ των 2 μεθόδων δεν είναι τόσο εμφανής όσο αναμέναμε.
Τα DataFrames αξιοποιούν το optimization και προσφέρουν γενικά μεγαλύτερη ταχύτητα, αν και στη συγκεκριμένη περίπτωση η διαφορά είναι μικρή.
Αντιθέτως, τα RDD προσφέρουν μεγαλύτερη ευελιξία, αφού είναι low level, στην επεξεργασία των δεδομένων. 

Πιθανόν το μέγεθος των δεδομένων να μην είναι αρκετά μεγάλο ώστε να αρχίσει να φαίνεται μια ουσιαστική διαφορά.

In [15]:
print(f"""Time taken for DataFrame API: {time_df}.
Time taken for RDD API: {time_rdd}
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for DataFrame API: 5.364398956298828.
Time taken for RDD API: 6.302131414413452

## Query 2

### α) Να υλοποιηθεί το __Query 2__ χρησιμοποιώντας τα DataFrame και SQL APIs. Να αναφέρετε και να συγκρίνετε τους χρόνους εκτέλεσης μεταξύ των δύο υλοποιήσεων.

#### DataFrame API

In [16]:
# Create the spark session

from pyspark.sql import SparkSession

spark = spark.newSession() \
    .builder \
    .appName("Query 2") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
crimes_df = spark.read.csv(fcrime, header=True, schema=crimes_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')
stations_df = spark.read.csv(fstations, header=True, schema=stations_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Start timer
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Create table yearly_precincts

yearly_precincts_df = crimes_df.join(
    stations_df,
    crimes_df.AREA.cast("int") == stations_df.FID
).groupBy(
    year(crimes_df.DateRptd).alias("year"),
    stations_df.DIVISION.alias("precinct")
).agg(
    sum(when(col('Status') != "IC", 1).otherwise(0)).alias("closed_cases"),
    count("*").alias("total_cases"),
    (
        sum(when(col('Status') != "IC", 1).otherwise(0)) * 100.0 / count("*")
    ).alias("closed_case_rate")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Create table ranked precincts

windowSpec = Window.partitionBy('year').orderBy(col('closed_case_rate').desc())

ranked_precincts_df = yearly_precincts_df.withColumn(
    'ranking', row_number().over(windowSpec)
).select(
    col('year'),
    col('precinct'),
    col('closed_case_rate'),
    col('ranking')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Create table rsults

results_df = ranked_precincts_df.filter(
    col('ranking') <= 3
).select(
    'year',
    'precinct',
    'closed_case_rate',
    'ranking'
).orderBy('year','ranking')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Final results:

results_df.show()
time_end = time.time()
df_time = time_end - time_start
print(f'Time taken since creation of DF spark session to completion: {df_time:.2f} seconds')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+------------------+-------+
|year|  precinct|  closed_case_rate|ranking|
+----+----------+------------------+-------+
|2010| SOUTHEAST|32.947355855318136|      1|
|2010|DEVONSHIRE|31.962706191728426|      2|
|2010| SOUTHWEST| 29.63203463203463|      3|
|2011|DEVONSHIRE|35.212167689161554|      1|
|2011| SOUTHEAST|32.511779630300836|      2|
|2011| SOUTHWEST| 28.65220520201501|      3|
|2012|DEVONSHIRE|34.414818310523835|      1|
|2012| SOUTHEAST|  32.9464181029429|      2|
|2012| SOUTHWEST|29.815133276010318|      3|
|2013|DEVONSHIRE| 33.52812271731191|      1|
|2013| SOUTHEAST| 32.08287360549222|      2|
|2013| SOUTHWEST|29.164224592662055|      3|
|2014|HOLLENBECK| 31.80567315834039|      1|
|2014|  WILSHIRE|31.311989956057754|      2|
|2014|  FOOTHILL|31.162790697674417|      3|
|2015|HOLLENBECK|32.641346981727736|      1|
|2015|  WILSHIRE|30.275974025974026|      2|
|2015|  FOOTHILL|30.179460678380156|      3|
|2016|HOLLENBECK|31.880755720117726|      1|
|2016|  WI

#### SQL API

In [23]:
# Start timer
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Crimes Table

crimes_schema_sql = StructType([
    StructField("DR_NO", StringType()),
    StructField("DateRptd", StringType()),
    StructField("DATEOCC", StringType()),
    StructField("TIMEOCC", StringType()),
    StructField("AREA", StringType()),
    StructField("AREANAME", StringType()),
    StructField("RptDistNo", StringType()),
    StructField("Part", IntegerType()),
    StructField("CrmCd", StringType()),
    StructField("CrmCdDesc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("VictAge", StringType()),
    StructField("VictSex", StringType()),
    StructField("VictDescent", StringType()),
    StructField("PremisCd", StringType()),
    StructField("PremisDesc", StringType()),
    StructField("WeaponUsedCd", StringType()),
    StructField("WeaponDesc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("CrmCd1", StringType()),
    StructField("CrmCd2", StringType()),
    StructField("CrmCd3", StringType()),
    StructField("CrmCd4", StringType()),
    StructField("LOCATION", StringType()),
    StructField("CrossStreet", StringType()),
    StructField("LAT", FloatType()),
    StructField("LON", FloatType()),
])

crimes_df = spark.read.format('csv') \
    .options(header='true', dateFormat='MM/dd/yyyy hh:mm:ss a') \
    .schema(crimes_schema_sql) \
    .load(fcrime)

crimes_df = crimes_df.withColumn("DateRptd", to_timestamp("DateRptd", "MM/dd/yyyy hh:mm:ss a")) \
                     .withColumn("DATEOCC", to_timestamp("DATEOCC", "MM/dd/yyyy hh:mm:ss a"))

crimes_df.createOrReplaceTempView("crimes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Stations Table

stations_schema_sql = StructType([
    StructField("X", FloatType()),
    StructField("Y", FloatType()),
    StructField("FID", IntegerType()),
    StructField("DIVISION", StringType()),
    StructField("LOCATION", StringType()),
    StructField("PREC", IntegerType()),
])

stations_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(stations_schema_sql) \
    .load(fstations)

stations_df.createOrReplaceTempView("stations")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
query2_sql = """WITH YearlyPrecinctStats AS ( 
    SELECT 
        YEAR(c.DateRptd) AS year,
        s.DIVISION AS precinct,
        COUNT(*) AS total_cases,
        SUM(CASE WHEN c.Status != 'IC' THEN 1 ELSE 0 END) AS closed_cases,
        SUM(CASE WHEN c.Status != 'IC' THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS closed_case_rate
    FROM crimes c
    JOIN stations s
        ON CAST(c.AREA as INTEGER) = s.FID
    GROUP BY YEAR(c.DateRptd), s.DIVISION
    ),
    rankedPrecincts AS (
        SELECT
            year,
            precinct,
            closed_case_rate,
            ROW_NUMBER() OVER (PARTITION BY year ORDER BY closed_case_rate DESC) AS ranking
        FROM YearlyPrecinctStats
    )
    SELECT
        year,
        precinct,
        closed_case_rate,
        ranking
    FROM RankedPrecincts
    WHERE ranking <= 3
    ORDER BY year, ranking;"""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Explanation:__ 

YearlyPrecinctStats
- We need to group our data by year and department
- Keep a count of all cases and a count of closed cases
- Create the rate as a percentage

RankedPrecincts
- From YearlyPrecinctStats keep: year, precinct and closed_case_rate
- We will need to create the ranking based on the closed cases rate of each department
- For each year assign a ranking (starting at 1) in descending order

Notes:

The symbol '#' is not supported in SQL so it was changed to ranking

In [27]:
spark.sql(query2_sql).show()
time_end = time.time()
sql_time = time_end - time_start
print(f"Time taken since creation of spark session to query completion: {sql_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+-----------------+-------+
|year|  precinct| closed_case_rate|ranking|
+----+----------+-----------------+-------+
|2010| SOUTHEAST|32.94735585531813|      1|
|2010|DEVONSHIRE|31.96270619172842|      2|
|2010| SOUTHWEST|29.63203463203463|      3|
|2011|DEVONSHIRE|35.21216768916155|      1|
|2011| SOUTHEAST|32.51177963030083|      2|
|2011| SOUTHWEST|28.65220520201501|      3|
|2012|DEVONSHIRE|34.41481831052383|      1|
|2012| SOUTHEAST|32.94641810294290|      2|
|2012| SOUTHWEST|29.81513327601032|      3|
|2013|DEVONSHIRE|33.52812271731191|      1|
|2013| SOUTHEAST|32.08287360549222|      2|
|2013| SOUTHWEST|29.16422459266206|      3|
|2014|HOLLENBECK|31.80567315834039|      1|
|2014|  WILSHIRE|31.31198995605775|      2|
|2014|  FOOTHILL|31.16279069767442|      3|
|2015|HOLLENBECK|32.64134698172773|      1|
|2015|  WILSHIRE|30.27597402597403|      2|
|2015|  FOOTHILL|30.17946067838016|      3|
|2016|HOLLENBECK|31.88075572011773|      1|
|2016|  WILSHIRE|31.547987616099

#### Συμπεράσματα

In [28]:
print(f'''Time taken for DataFrame API: {df_time:.2f}
Time taken for SQL API: {sql_time:.2f}''')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for DataFrame API: 4.38
Time taken for SQL API: 3.38

Παρατηρούμε ότι οι χρόνοι εκτέλεσης δεν παρουσιάζουν μεγάλες αποκλίσεις.
Παραδόξως, οι χρόνοι εκτέλεσης για το SQL API είναι συνήθως σταθεροί για όσες φορές τρέξουμε τον κώδικα, ενώ στο DataFrame API παρατηρούμε μεγάλη απόκλιση μεταξύ των τιμών για πολλαπλές εκτελέσεις του κώδικα.

### β) Να γράψετε κώδικα Spark που μετατρέπει το κυρίως data set σε parquet file format και αποθηκεύει ένα μοναδικό .parquet αρχείο στο S3 bucket της ομάδας σας. Επιλέξτε μία από τις δύο υλοποιήσεις του υποερωτήματος α) (DataFrame ή SQL) και συγκρίνετε τους χρόνους εκτέλεσης της εφαρμογής σας όταν τα δεδομένα εισάγονται σαν .csv και σαν .parquet.

In [29]:
# Create the spark session

from pyspark.sql import SparkSession

spark = spark.newSession() \
    .builder \
    .config("spark.hadoop.fs.hdfs.impl.disable.cache", "true") \
    .config("spark.hadoop.io.native.lib.available", "false") \
    .appName("Query 2 Part 2") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
crimes_df = spark.read.csv(fcrime,header=True, inferSchema=True)
crimes_df.write.mode('overwrite').parquet(fcrime_parq)

stations_df = spark.read.csv(fstations,header=True, inferSchema=True)
stations_df.write.mode('overwrite').parquet(fstations_parq)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Θα χρησιμοποιήσουμε το DataFrame:

In [31]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
crimes_df = spark.read.parquet(fcrime_parq)
stations_df = spark.read.parquet(fstations_parq)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
crimes_df = crimes_df.withColumn('Date Rptd', to_timestamp('Date Rptd', 'MM/dd/yyyy hh:mm:ss a'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Create table yearly_precincts

yearly_precincts_df = crimes_df.join(
    stations_df,
    col('AREA ').cast("int") == stations_df.FID
).groupBy(
    year(col('Date Rptd')).alias("year"),
    stations_df.DIVISION.alias("precinct")
).agg(
    sum(when(col('Status') != "IC", 1).otherwise(0)).alias("closed_cases"),
    count("*").alias("total_cases"),
    (
        sum(when(col('Status') != "IC", 1).otherwise(0)) * 100.0 / count("*")
    ).alias("closed_case_rate")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Create table ranked precincts

windowSpec = Window.partitionBy('year').orderBy(col('closed_case_rate').desc())

ranked_precincts_df = yearly_precincts_df.withColumn(
    'ranking', row_number().over(windowSpec)
).select(
    col('year'),
    col('precinct'),
    col('closed_case_rate'),
    col('ranking')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Create table results

results_df = ranked_precincts_df.filter(
    col('ranking') <= 3
).select(
    'year',
    'precinct',
    'closed_case_rate',
    'ranking'
).orderBy('year','ranking')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Final results:

results_df.show()
time_end = time.time()
df_time_parq = time_end - time_start
print(f'Time taken for DF with parquet to completion: {df_time_parq:.2f} seconds')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+------------------+-------+
|year|  precinct|  closed_case_rate|ranking|
+----+----------+------------------+-------+
|2010| SOUTHEAST|32.947355855318136|      1|
|2010|DEVONSHIRE|31.962706191728426|      2|
|2010| SOUTHWEST| 29.63203463203463|      3|
|2011|DEVONSHIRE|35.212167689161554|      1|
|2011| SOUTHEAST|32.511779630300836|      2|
|2011| SOUTHWEST| 28.65220520201501|      3|
|2012|DEVONSHIRE|34.414818310523835|      1|
|2012| SOUTHEAST|  32.9464181029429|      2|
|2012| SOUTHWEST|29.815133276010318|      3|
|2013|DEVONSHIRE| 33.52812271731191|      1|
|2013| SOUTHEAST| 32.08287360549222|      2|
|2013| SOUTHWEST|29.164224592662055|      3|
|2014|HOLLENBECK| 31.80567315834039|      1|
|2014|  WILSHIRE|31.311989956057754|      2|
|2014|  FOOTHILL|31.162790697674417|      3|
|2015|HOLLENBECK|32.641346981727736|      1|
|2015|  WILSHIRE|30.275974025974026|      2|
|2015|  FOOTHILL|30.179460678380156|      3|
|2016|HOLLENBECK|31.880755720117726|      1|
|2016|  WI

Χρησιμοποιώντας τα parquet δεδομένα, παρατηρούμε σταθερά μια αρκετά μεγάλη βελτίωση στην ταχύτητα.

## Query 3

Να υλοποιηθεί το Query 3 χρησιμοποιώντας DataFrame ή SQL API. Χρησιμοποιήστε τις μεθόδους hint & explain για να βρείτε ποιες στρατηγικές join χρησιμοποιεί ο catalyst optimizer.
Πειραματιστείτε αναγκάζοντας το Spark να χρησιμοποιήσει διαφορετικές στρατηγικές (μεταξύ
των BROADCAST, MERGE, SHUFFLE_HASH, SHUFFLE_REPLICATE_NL) και σχολιάστε τα αποτελέσματα
που παρατηρείτε. Ποιά (ή ποιές) από τις διαθέσιμες στρατηγικές join του Spark είναι καταλληλότερη(ες) και γιατί;

In [38]:
from sedona.spark import *

sedona = SedonaContext.create(spark)

blocks_df = sedona.read.format('geojson') \
    .option('multiLine','true').load(fgeo) \
    .selectExpr('explode(features) as features') \
    .select('features.*')

flattened_df = blocks_df.select(
    [col(f'properties.{col_name}').alias(col_name) for col_name in \
    blocks_df.schema['properties'].dataType.fieldNames()] + ['geometry']) \
    .drop('properties').drop('type')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
income_schema = StructType([
    StructField('ZipCode', IntegerType()),
    StructField('Community', StringType()),
    StructField('Income', StringType())
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
income_df = spark.read.csv(fincome, header=True, schema=income_schema)

# Remove the $ character
income_df = income_df.withColumn(
    'Income', 
    regexp_replace(col('Income'), r"[$,]", "").cast('float')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
crimes_df = spark.read.csv(fcrime, header=True, schema=crimes_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')
stations_df = spark.read.csv(fstations, header=True, schema=stations_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
la_flattened_df = flattened_df.filter(col('CITY') == 'Los Angeles')
crimes_df = crimes_df.filter(
    (col('LAT') != 0) & (col('LON') != 0)
)

time_start = time.time()

result_df = crimes_df.withColumn('geom', ST_Point(col('LON'), col('LAT')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
income_per_area_df = income_df.join(la_flattened_df, col('ZipCode') == la_flattened_df['ZCTA10'])

income_per_area_df.explain()

# Table for Query 4
q4 = result_df.join(
    income_per_area_df,
    ST_Within(result_df['geom'], la_flattened_df['geometry'])
)

result_df = result_df.join(
    income_per_area_df,
    ST_Within(result_df['geom'], la_flattened_df['geometry']),
    how='inner'
).groupBy('COMM').agg(
    (sum('Income') / sum('POP_2010')).alias('IncomePerPerson'),
    (count('*') / sum('POP_2010')).alias('CrimesPerPerson')
).orderBy(col('IncomePerPerson').asc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [ZipCode#1298], [cast(ZCTA10#1194 as int)], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=1275]
   :  +- Project [ZipCode#1298, Community#1299, cast(regexp_replace(Income#1300, [$,], , 1) as float) AS Income#1304]
   :     +- Filter isnotnull(ZipCode#1298)
   :        +- FileScan csv [ZipCode#1298,Community#1299,Income#1300] Batched: false, DataFilters: [isnotnull(ZipCode#1298)], Format: CSV, Location: InMemoryFileIndex(1 paths)[s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ZipCode)], ReadSchema: struct<ZipCode:int,Community:string,Income:string>
   +- Project [features#1161.properties.BG10 AS BG10#1170, features#1161.properties.BG10FIP10 AS BG10FIP10#1171, features#1161.properties.BG12 AS BG12#1172, features#1161.properties.CB10 AS CB10#1173, fea

In [44]:
result_df.show()

result_df.explain()

time_end = time.time()
timeQ3Base = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|              COMM|   IncomePerPerson|     CrimesPerPerson|
+------------------+------------------+--------------------+
|          Westlake|46.097391668899746|0.001780055400923...|
|     Panorama City|59.679510369299315|0.001597238227038...|
|     Baldwin Hills| 78.72397418970363|0.002119263022880266|
| Little Bangladesh| 79.19429343714515|0.002037585700718...|
|   Wilshire Center| 79.91559362366671|0.002108441605087...|
|   University Park| 84.82775008029655|0.003672160928616...|
|         Thai Town| 92.49907318881583|0.001792791417556...|
|Wholesale District| 95.19512628794847|0.004395147375951249|
|     Vermont Vista| 95.96195522120188|0.003189405043223...|
|        South Park| 96.07853726620282|0.003380633455086159|
|        Pico-Union|100.70963557402804|0.003261075176157...|
|    Toluca Terrace|102.38366107576634|0.002111046847888953|
|        West Adams|104.16825869722096|0.003524301019893139|
|          Van Nuys|104.

In [45]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
result_df = crimes_df.withColumn('geom', ST_Point(col('LON'), col('LAT')))

income_per_area_df = income_df.join(
    la_flattened_df.hint('broadcast'), 
    col('ZipCode') == la_flattened_df['ZCTA10'],
    how='inner'
)

result_df = result_df.join(
    income_per_area_df.hint('broadcast'),
    ST_Within(result_df['geom'], la_flattened_df['geometry']),
    how='inner'
).groupBy('COMM').agg(
    (sum('Income') / sum('POP_2010')).alias('IncomePerPerson'),
    (count('*') / sum('POP_2010')).alias('CrimesPerPerson')
).orderBy(col('IncomePerPerson').asc())

result_df.explain(True)

result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
'Sort ['IncomePerPerson ASC NULLS FIRST], true
+- Aggregate [COMM#1177], [COMM#1177, (sum(Income#1304) / cast(sum(POP_2010#1186L) as double)) AS IncomePerPerson#2076, (cast(count(1) as double) / cast(sum(POP_2010#1186L) as double)) AS CrimesPerPerson#2079]
   +- Join Inner,  **org.apache.spark.sql.sedona_sql.expressions.ST_Within**
      :- Project [DR_NO#1309, DateRptd#1310, DATEOCC#1311, TIMEOCC#1312, AREA#1313, AREANAME#1314, RptDistNo#1315, Part#1316, CrmCd#1317, Crm Cd Desc#1318, Mocodes#1319, Vict Age#1320, VictSex#1321, VictDescent#1322, PremisCd#1323, PremisDesc#1324, WeaponUsedCd#1325, WeaponDesc#1326, Status#1327, Status Desc#1328, CrmCd1#1329, CrmCd2#1330, CrmCd3#1331, CrmCd4#1332, ... 5 more fields]
      :  +- Filter (NOT (LAT#1335 = cast(0 as float)) AND NOT (LON#1336 = cast(0 as float)))
      :     +- Relation [DR_NO#1309,DateRptd#1310,DATEOCC#1311,TIMEOCC#1312,AREA#1313,AREANAME#1314,RptDistNo#1315,Part#1316,CrmCd#1317,Crm Cd Desc#1318,Mocodes

In [47]:
time_end = time.time()
timeQ3Broadcast = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
result_df_merge = crimes_df.withColumn('geom', ST_Point(col('LON'), col('LAT')))

result_df_merge = result_df_merge.join(
    income_per_area_df.hint('merge'),
    ST_Within(result_df_merge['geom'], la_flattened_df['geometry']),
    how='inner'
).groupBy('COMM').agg(
    (sum('Income') / sum('POP_2010')).alias('IncomePerPerson'),
    (count('*') / sum('POP_2010')).alias('CrimesPerPerson')
).orderBy(col('IncomePerPerson').asc())

result_df_merge.show()

result_df_merge.explain(True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|              COMM|   IncomePerPerson|     CrimesPerPerson|
+------------------+------------------+--------------------+
|          Westlake|46.097391668899746|0.001780055400923...|
|     Panorama City|59.679510369299315|0.001597238227038...|
|     Baldwin Hills| 78.72397418970363|0.002119263022880266|
| Little Bangladesh| 79.19429343714515|0.002037585700718...|
|   Wilshire Center| 79.91559362366671|0.002108441605087...|
|   University Park| 84.82775008029655|0.003672160928616...|
|         Thai Town| 92.49907318881583|0.001792791417556...|
|Wholesale District| 95.19512628794847|0.004395147375951249|
|     Vermont Vista| 95.96195522120188|0.003189405043223...|
|        South Park| 96.07853726620282|0.003380633455086159|
|        Pico-Union|100.70963557402804|0.003261075176157...|
|    Toluca Terrace|102.38366107576634|0.002111046847888953|
|        West Adams|104.16825869722096|0.003524301019893139|
|          Van Nuys|104.

In [50]:
time_end = time.time()
timeQ3Merge = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
result_df = crimes_df.withColumn('geom', ST_Point(col('LON'), col('LAT')))

income_per_area_df = income_df.join(
    la_flattened_df.hint('shuffle_hash'), 
    col('ZipCode') == la_flattened_df['ZCTA10'],
    how='inner'
)

result_df = result_df.join(
    income_per_area_df.hint('shuffle_hush'),
    ST_Within(result_df['geom'], la_flattened_df['geometry']),
    how='inner'
).groupBy('COMM').agg(
    (sum('Income') / sum('POP_2010')).alias('IncomePerPerson'),
    (count('*') / sum('POP_2010')).alias('CrimesPerPerson')
).orderBy(col('IncomePerPerson').asc())

result_df.explain(True)

result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
'Sort ['IncomePerPerson ASC NULLS FIRST], true
+- Aggregate [COMM#1177], [COMM#1177, (sum(Income#1304) / cast(sum(POP_2010#1186L) as double)) AS IncomePerPerson#2658, (cast(count(1) as double) / cast(sum(POP_2010#1186L) as double)) AS CrimesPerPerson#2661]
   +- Join Inner,  **org.apache.spark.sql.sedona_sql.expressions.ST_Within**
      :- Project [DR_NO#1309, DateRptd#1310, DATEOCC#1311, TIMEOCC#1312, AREA#1313, AREANAME#1314, RptDistNo#1315, Part#1316, CrmCd#1317, Crm Cd Desc#1318, Mocodes#1319, Vict Age#1320, VictSex#1321, VictDescent#1322, PremisCd#1323, PremisDesc#1324, WeaponUsedCd#1325, WeaponDesc#1326, Status#1327, Status Desc#1328, CrmCd1#1329, CrmCd2#1330, CrmCd3#1331, CrmCd4#1332, ... 5 more fields]
      :  +- Filter (NOT (LAT#1335 = cast(0 as float)) AND NOT (LON#1336 = cast(0 as float)))
      :     +- Relation [DR_NO#1309,DateRptd#1310,DATEOCC#1311,TIMEOCC#1312,AREA#1313,AREANAME#1314,RptDistNo#1315,Part#1316,CrmCd#1317,Crm Cd Desc#1318,Mocodes

In [53]:
time_end = time.time()
timeQ3ShuffleHash = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
result_df = crimes_df.withColumn('geom', ST_Point(col('LON'), col('LAT')))

income_per_area_df = income_df.join(
    la_flattened_df.hint('shuffle_replicate_nl'), 
    col('ZipCode') == la_flattened_df['ZCTA10'],
    how='inner'
)

result_df = result_df.join(
    income_per_area_df.hint('shuffle_replicate_nl'),
    ST_Within(result_df['geom'], la_flattened_df['geometry']),
    how='inner'
).groupBy('COMM').agg(
    (sum('Income') / sum('POP_2010')).alias('IncomePerPerson'),
    (count('*') / sum('POP_2010')).alias('CrimesPerPerson')
).orderBy(col('IncomePerPerson').asc())

result_df.explain(True)

result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
'Sort ['IncomePerPerson ASC NULLS FIRST], true
+- Aggregate [COMM#1177], [COMM#1177, (sum(Income#1304) / cast(sum(POP_2010#1186L) as double)) AS IncomePerPerson#2976, (cast(count(1) as double) / cast(sum(POP_2010#1186L) as double)) AS CrimesPerPerson#2979]
   +- Join Inner,  **org.apache.spark.sql.sedona_sql.expressions.ST_Within**
      :- Project [DR_NO#1309, DateRptd#1310, DATEOCC#1311, TIMEOCC#1312, AREA#1313, AREANAME#1314, RptDistNo#1315, Part#1316, CrmCd#1317, Crm Cd Desc#1318, Mocodes#1319, Vict Age#1320, VictSex#1321, VictDescent#1322, PremisCd#1323, PremisDesc#1324, WeaponUsedCd#1325, WeaponDesc#1326, Status#1327, Status Desc#1328, CrmCd1#1329, CrmCd2#1330, CrmCd3#1331, CrmCd4#1332, ... 5 more fields]
      :  +- Filter (NOT (LAT#1335 = cast(0 as float)) AND NOT (LON#1336 = cast(0 as float)))
      :     +- Relation [DR_NO#1309,DateRptd#1310,DATEOCC#1311,TIMEOCC#1312,AREA#1313,AREANAME#1314,RptDistNo#1315,Part#1316,CrmCd#1317,Crm Cd Desc#1318,Mocodes

In [56]:
time_end = time.time()
timeQ3ShuffleRepl = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
print(f"""Times for each hint:
Base: {timeQ3Base:.2f} sec
Broadcast: {timeQ3Broadcast:.2f} sec
Merge: {timeQ3Merge:.2f} sec
Shuffle Hash: {timeQ3ShuffleHash:.2f} sec
Shuffle Replicate: {timeQ3ShuffleRepl:.2f} sec
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Times for each hint:
Base: 20.62 sec
Broadcast: 17.29 sec
Merge: 17.29 sec
Shuffle Hash: 17.30 sec
Shuffle Replicate: 17.29 sec

## Query 4

Να υλοποιηθεί το Query 4 χρησιμοποιώντας το DataFrame ή SQL API. Να εκτελέσετε την υλοποίησή σας εφαρμόζοντας κλιμάκωση στο σύνολο των υπολογιστικών πόρων που θα χρησιμοποιήσετε: Συγκεκριμένα, καλείστε να εκτελέστε την υλοποίησή σας σε 2 executors με τα ακόλουθα configurations:

- 1 core/2 GB memory
- 2 cores/4GB memory
- 4 cores/8GB memory

Σχολιάστε τα αποτελέσματα.

In [58]:
crimes_df = spark.read.csv(fcrime, header=True, schema=crimes_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')
stations_df = spark.read.csv(fstations, header=True, schema=stations_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1 core / 2 GB

In [59]:
# Create the spark session

from pyspark.sql import SparkSession

spark = spark.newSession() \
    .builder \
    .appName("Query 4 Part 1") \
    .config('spark.executor.instances','2') \
    .config('spark.executor.cores', '1') \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
code_df = spark.read.csv(fcodes, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
highest_income_df = result_df.orderBy(col('IncomePerPerson').desc()).limit(3) # Using the table in Query 3
lowest_income_df = result_df.orderBy('IncomePerPerson').limit(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
crimes_2015_df = q4.withColumn('DATEOCC', to_timestamp('DATEOCC','MM/dd/yyyy hh:mm:ss a'))
crimes_2015_df = crimes_2015_df.filter(year(col('DATEOCC')) == 2015)

high_income_crimes = crimes_2015_df.join(
    highest_income_df,
    crimes_2015_df['COMM'] == highest_income_df['COMM']
)

low_income_crimes = crimes_2015_df.join(
    lowest_income_df,
    crimes_2015_df['COMM'] == lowest_income_df['COMM']
)

high_income_race = high_income_crimes.join(
    code_df,
    high_income_crimes['VictDescent'] == code_df['Vict Descent']
)

low_income_race = low_income_crimes.join(
    code_df,
    low_income_crimes['VictDescent'] == code_df['Vict Descent']
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
results_high_df = high_income_race.groupBy('Vict Descent Full').count().alias('Count').orderBy(col('Count').desc())

results_high_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|   Vict Descent Full|count|
+--------------------+-----+
|               White|  150|
|               Other|   29|
|Hispanic/Latin/Me...|   23|
|               Black|   16|
|         Other Asian|    8|
|             Unknown|    5|
+--------------------+-----+

In [65]:
results_low_df = low_income_race.groupBy('Vict Descent Full').count().alias('Count').orderBy(col('Count').desc())

results_low_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|   Vict Descent Full|count|
+--------------------+-----+
|Hispanic/Latin/Me...| 4435|
|               Black| 2296|
|               White|  956|
|               Other|  931|
|         Other Asian|  198|
|             Unknown|  174|
|              Korean|   38|
|            Filipino|   21|
|           Guamanian|    3|
|             Chinese|    2|
|American Indian/A...|    2|
|            Japanese|    2|
|    Pacific Islander|    2|
+--------------------+-----+

In [66]:
time_end = time.time()
timeQ4_1 = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2 cores / 4 GB

In [67]:
# Create the spark session

from pyspark.sql import SparkSession

spark = spark.newSession() \
    .builder \
    .appName("Query 4 Part 2") \
    .config('spark.executor.instances','2') \
    .config('spark.executor.cores', '2') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
code_df = spark.read.csv(fcodes, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
highest_income_df = result_df.orderBy(col('IncomePerPerson').desc()).limit(3) # Using the table in Query 3
lowest_income_df = result_df.orderBy('IncomePerPerson').limit(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
crimes_2015_df = q4.withColumn('DATEOCC', to_timestamp('DATEOCC','MM/dd/yyyy hh:mm:ss a'))
crimes_2015_df = crimes_2015_df.filter(year(col('DATEOCC')) == 2015)

high_income_crimes = crimes_2015_df.join(
    highest_income_df,
    crimes_2015_df['COMM'] == highest_income_df['COMM']
)

low_income_crimes = crimes_2015_df.join(
    lowest_income_df,
    crimes_2015_df['COMM'] == lowest_income_df['COMM']
)

high_income_race = high_income_crimes.join(
    code_df,
    high_income_crimes['VictDescent'] == code_df['Vict Descent']
)

low_income_race = low_income_crimes.join(
    code_df,
    low_income_crimes['VictDescent'] == code_df['Vict Descent']
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
results_high_df = high_income_race.groupBy('Vict Descent Full').count().alias('Count').orderBy(col('Count').desc())

results_high_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|   Vict Descent Full|count|
+--------------------+-----+
|               White|  150|
|               Other|   29|
|Hispanic/Latin/Me...|   23|
|               Black|   16|
|         Other Asian|    8|
|             Unknown|    5|
+--------------------+-----+

In [73]:
results_low_df = low_income_race.groupBy('Vict Descent Full').count().alias('Count').orderBy(col('Count').desc())

results_low_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|   Vict Descent Full|count|
+--------------------+-----+
|Hispanic/Latin/Me...| 4435|
|               Black| 2296|
|               White|  956|
|               Other|  931|
|         Other Asian|  198|
|             Unknown|  174|
|              Korean|   38|
|            Filipino|   21|
|           Guamanian|    3|
|            Japanese|    2|
|    Pacific Islander|    2|
|             Chinese|    2|
|American Indian/A...|    2|
+--------------------+-----+

In [74]:
time_end = time.time()
timeQ4_2 = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4 cores / 8 GB

In [75]:
# Create the spark session

from pyspark.sql import SparkSession

spark = spark.newSession() \
    .builder \
    .appName("Query 4 Part 3") \
    .config('spark.executor.instances','2') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
time_start = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
code_df = spark.read.csv(fcodes, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
highest_income_df = result_df.orderBy(col('IncomePerPerson').desc()).limit(3) # Using the table in Query 3
lowest_income_df = result_df.orderBy('IncomePerPerson').limit(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
crimes_2015_df = q4.withColumn('DATEOCC', to_timestamp('DATEOCC','MM/dd/yyyy hh:mm:ss a'))
crimes_2015_df = crimes_2015_df.filter(year(col('DATEOCC')) == 2015)

high_income_crimes = crimes_2015_df.join(
    highest_income_df,
    crimes_2015_df['COMM'] == highest_income_df['COMM']
)

low_income_crimes = crimes_2015_df.join(
    lowest_income_df,
    crimes_2015_df['COMM'] == lowest_income_df['COMM']
)

high_income_race = high_income_crimes.join(
    code_df,
    high_income_crimes['VictDescent'] == code_df['Vict Descent']
)

low_income_race = low_income_crimes.join(
    code_df,
    low_income_crimes['VictDescent'] == code_df['Vict Descent']
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
results_high_df = high_income_race.groupBy('Vict Descent Full').count().alias('Count').orderBy(col('Count').desc())

results_high_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|   Vict Descent Full|count|
+--------------------+-----+
|               White|  150|
|               Other|   29|
|Hispanic/Latin/Me...|   23|
|               Black|   16|
|         Other Asian|    8|
|             Unknown|    5|
+--------------------+-----+

In [81]:
results_low_df = low_income_race.groupBy('Vict Descent Full').count().alias('Count').orderBy(col('Count').desc())

results_low_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|   Vict Descent Full|count|
+--------------------+-----+
|Hispanic/Latin/Me...| 4435|
|               Black| 2296|
|               White|  956|
|               Other|  931|
|         Other Asian|  198|
|             Unknown|  174|
|              Korean|   38|
|            Filipino|   21|
|           Guamanian|    3|
|            Japanese|    2|
|             Chinese|    2|
|    Pacific Islander|    2|
|American Indian/A...|    2|
+--------------------+-----+

In [82]:
time_end = time.time()
timeQ4_3 = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
print(f"""Times taken for each settings:
- 1 core / 2 GB: {timeQ4_1:.2f} sec
- 2 core / 4 GB: {timeQ4_2:.2f} sec
- 4 core / 8 GB: {timeQ4_3:.2f} sec
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Times taken for each settings:
- 1 core / 2 GB: 59.81 sec
- 2 core / 4 GB: 59.80 sec
- 4 core / 8 GB: 57.80 sec

## Query 5

In [84]:
crimes_df = spark.read.csv(fcrime, header=True, schema=crimes_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')
stations_df = spark.read.csv(fstations, header=True, schema=stations_schema, dateFormat='MM/dd/yyyy hh:mm:ss a')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2 Executors x 4 cores / 8 GB

In [85]:
from pyspark.sql.functions import avg, desc
from sedona.spark import *

spark = spark.newSession().builder \
    .appName("Query 5 Part 1") \
    .config('spark.executor.instances', '2') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

# Initialize SedonaContext
sedona = SedonaContext.create(spark)

stations_geom_df = stations_df.withColumn('StationsGeom', ST_Point(col('Y'), col('X')))

crimes_geom_df = crimes_df.filter((col('LAT') != 0) & (col('LON') != 0)) \
    .withColumn('CrimesGeom', ST_Point(col('LAT'), col('LON')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
# Start the timer
time_start = time.time()

# Perform a cross join to calculate distances between all crimes and stations
distance_df = crimes_geom_df.crossJoin(stations_geom_df) \
    .withColumn(
        'distance',
        (ST_DistanceSphere('CrimesGeom', 'StationsGeom') / 1000)
    )

# Find the closest station for each crime
window_spec = Window.partitionBy("DR_NO").orderBy("distance")
closest_station_df = distance_df.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1)  # Keep only the closest station

# Aggregate results by station
result_df = closest_station_df.groupBy("Division") \
    .agg(
        avg("distance").alias("avg_distance"),  # Average distance of crimes to the station
        count("*").alias("num_crimes")  # Total crimes closest to the station
    ) \
    .orderBy(desc("num_crimes"))  # Sort by number of crimes in descending order

# Display the results
result_df.show(21)

# End timer
time_end = time.time()
timeQ5_1 = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+----------+
|        Division|      avg_distance|num_crimes|
+----------------+------------------+----------+
|         OLYMPIC|1.5399445462398453|    153479|
|        WILSHIRE|2.2489600333972377|    153129|
|        VAN NUYS| 2.116039225369102|    141765|
|       SOUTHWEST|1.7235245838416475|    135620|
| NORTH HOLLYWOOD|2.2360621498035207|    129372|
|       HOLLYWOOD| 1.762258175523237|    112887|
|     77TH STREET|1.2236326775408635|    111430|
|      HOLLENBECK| 2.614000796608531|    111427|
|         TOPANGA|2.3733886897811036|    108598|
|         RAMPART|1.0616486612137963|     99284|
|       SOUTHEAST|1.9159889369204524|     95160|
|         PACIFIC| 2.777580705804232|     92999|
|         CENTRAL|0.7766393931203249|     91676|
|WEST LOS ANGELES| 2.470498404763837|     89606|
|          HARBOR| 2.427327047090941|     88494|
|     WEST VALLEY|2.5187343779980047|     88196|
|         MISSION| 2.471253774754481|     82504|
|        FOOTHILL|2.

### 4 Executors x 2 cores / 4 GB

In [87]:
from pyspark.sql.functions import avg, desc
from sedona.spark import *

spark = spark.newSession().builder \
    .appName("Query 5 Part 2") \
    .config('spark.executor.instances', '4') \
    .config('spark.executor.cores', '2') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

# Initialize SedonaContext
sedona = SedonaContext.create(spark)

stations_geom_df = stations_df.withColumn('StationsGeom', ST_Point(col('Y'), col('X')))

crimes_geom_df = crimes_df.filter((col('LAT') != 0) & (col('LON') != 0)) \
    .withColumn('CrimesGeom', ST_Point(col('LAT'), col('LON')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
# Start the timer
time_start = time.time()

# Perform a cross join to calculate distances between all crimes and stations
distance_df = crimes_geom_df.crossJoin(stations_geom_df) \
    .withColumn(
        'distance',
        (ST_DistanceSphere('CrimesGeom', 'StationsGeom') / 1000)
    )

# Find the closest station for each crime
window_spec = Window.partitionBy("DR_NO").orderBy("distance")
closest_station_df = distance_df.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1)  # Keep only the closest station

# Aggregate results by station
result_df = closest_station_df.groupBy("Division") \
    .agg(
        avg("distance").alias("avg_distance"),  # Average distance of crimes to the station
        count("*").alias("num_crimes")  # Total crimes closest to the station
    ) \
    .orderBy(desc("num_crimes"))  # Sort by number of crimes in descending order

# Display the results
result_df.show(21)

# End timer
time_end = time.time()
timeQ5_2 = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+----------+
|        Division|      avg_distance|num_crimes|
+----------------+------------------+----------+
|         OLYMPIC|1.5399445462398456|    153479|
|        WILSHIRE|2.2489600333972377|    153129|
|        VAN NUYS| 2.116039225369102|    141765|
|       SOUTHWEST|1.7235245838416475|    135620|
| NORTH HOLLYWOOD|2.2360621498035207|    129372|
|       HOLLYWOOD| 1.762258175523237|    112887|
|     77TH STREET|1.2236326775408637|    111430|
|      HOLLENBECK|2.6140007966085306|    111427|
|         TOPANGA| 2.373388689781103|    108598|
|         RAMPART|1.0616486612137965|     99284|
|       SOUTHEAST|1.9159889369204524|     95160|
|         PACIFIC|2.7775807058042323|     92999|
|         CENTRAL| 0.776639393120325|     91676|
|WEST LOS ANGELES| 2.470498404763837|     89606|
|          HARBOR|  2.42732704709094|     88494|
|     WEST VALLEY|2.5187343779980047|     88196|
|         MISSION|2.4712537747544805|     82504|
|        FOOTHILL|2.

### 8 Executors x 1 cores / 2 GB

In [89]:
from pyspark.sql.functions import avg, desc
from sedona.spark import *

spark = spark.newSession().builder \
    .appName("Query 5 Part 3") \
    .config('spark.executor.instances', '8') \
    .config('spark.executor.cores', '1') \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

# Initialize SedonaContext
sedona = SedonaContext.create(spark)

stations_geom_df = stations_df.withColumn('StationsGeom', ST_Point(col('Y'), col('X')))

crimes_geom_df = crimes_df.filter((col('LAT') != 0) & (col('LON') != 0)) \
    .withColumn('CrimesGeom', ST_Point(col('LAT'), col('LON')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
# Start the timer
time_start = time.time()

# Perform a cross join to calculate distances between all crimes and stations
distance_df = crimes_geom_df.crossJoin(stations_geom_df) \
    .withColumn(
        'distance',
        (ST_DistanceSphere('CrimesGeom', 'StationsGeom') / 1000)
    )

# Find the closest station for each crime
window_spec = Window.partitionBy("DR_NO").orderBy("distance")
closest_station_df = distance_df.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1)  # Keep only the closest station

# Aggregate results by station
result_df = closest_station_df.groupBy("Division") \
    .agg(
        avg("distance").alias("avg_distance"),  # Average distance of crimes to the station
        count("*").alias("num_crimes")  # Total crimes closest to the station
    ) \
    .orderBy(desc("num_crimes"))  # Sort by number of crimes in descending order

# Display the results
result_df.show(21)

# End timer
time_end = time.time()
timeQ5_3 = time_end - time_start

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+----------+
|        Division|      avg_distance|num_crimes|
+----------------+------------------+----------+
|         OLYMPIC|1.5399445462398456|    153479|
|        WILSHIRE|2.2489600333972377|    153129|
|        VAN NUYS| 2.116039225369102|    141765|
|       SOUTHWEST| 1.723524583841648|    135620|
| NORTH HOLLYWOOD|2.2360621498035207|    129372|
|       HOLLYWOOD|1.7622581755232372|    112887|
|     77TH STREET|1.2236326775408637|    111430|
|      HOLLENBECK| 2.614000796608531|    111427|
|         TOPANGA|2.3733886897811036|    108598|
|         RAMPART|1.0616486612137963|     99284|
|       SOUTHEAST| 1.915988936920453|     95160|
|         PACIFIC|2.7775807058042323|     92999|
|         CENTRAL| 0.776639393120325|     91676|
|WEST LOS ANGELES|2.4704984047638368|     89606|
|          HARBOR|2.4273270470909414|     88494|
|     WEST VALLEY|2.5187343779980047|     88196|
|         MISSION|2.4712537747544805|     82504|
|        FOOTHILL|2.

In [91]:
print(f"""Times for each case:
Executors: 2, Cores: 4, Memory: 8 GB, Time: {timeQ5_1:.2f}
Executors: 4, Cores: 2, Memory: 4 GB, Time: {timeQ5_2:.2f}
Executors: 8, Cores: 1, Memory: 2 GB, Time: {timeQ5_3:.2f}
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Times for each case:
Executors: 2, Cores: 4, Memory: 8 GB, Time: 5.53
Executors: 4, Cores: 2, Memory: 4 GB, Time: 3.79
Executors: 8, Cores: 1, Memory: 2 GB, Time: 3.54